# Cleaning data for Technology Sector

# Read Data

In [1]:
import pandas as pd
options_data = pd.read_csv('../Data/Options_data_v2.csv')
#get Call data
index_names = options_data[options_data['OptionType'] == 'put'].index
options_data.drop(index_names, inplace=True)
#filter sector
other_sector = options_data[options_data['Sector'] != 'Technology'].index
options_data.drop(other_sector, inplace=True)
#remove NA
options_data.dropna(axis=0, inplace=True)
#reset index
options_data.reset_index(inplace=True)

In [2]:
options_data

,index,Unnamed: 0,OptionType,Time,Sym,Expiration,Strike,StockPrice,Date,Spent,Sector,ExpirationDate,ChainLocation,Spent/contract,Spent/share,break_even,Contract period in hrs,Contract period in days
0,0,0,call,09:30:00.0000000,TWLO,08/09-19 W,133.0,129.94,2019-08-09,35000,Technology,2019-08-09,OTM,350.0,3.5,136.5,6.499722,0
1,7,7,call,09:57:00.0000000,OSTK,08/09-19 W,25.0,23.66,2019-08-09,18000,Technology,2019-08-09,OTM,180.0,1.8,26.8,6.049722,0
2,8,8,call,10:02:00.0000000,HUYA,08-19 M,21.5,20.92,2019-08-09,73000,Technology,2019-08-16,OTM,730.0,7.3,28.8,173.966389,7
3,12,12,call,10:34:00.0000000,CLDR,08/23-19 W,7.5,6.97,2019-08-09,13000,Technology,2019-08-23,OTM,130.0,1.3,8.8,341.433056,14
4,15,15,call,10:42:00.0000000,NVDA,08-19 M,155.0,153.36,2019-08-09,344000,Technology,2019-08-16,OTM,3440.0,34.4,189.4,173.299722,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10370,43025,43025,call,13:07:46.0000000,IBM,11-17 M,149.0,148.47,2017-11-13,56000,Technology,2017-11-17,OTM,560.0,5.6,154.6,98.870278,4
10371,43029,43029,call,13:39:02.0000000,OLED,01-18 M,190.0,177.30,2017-11-13,1170000,Technology,2018-01-19,OTM,11700.0,117.0,307.0,1610.349167,67
10372,43034,43034,call,14:10:32.0000000,TWTR,01-18 M,20.0,20.17,2017-11-13,412000,Technology,2018-01-19,ITM,4120.0,41.2,61.2,1609.824167,67
10373,43039,43039,call,14:24:36.0000000,YNDX,01-18 M,32.0,31.66,2017-11-13,205000,Technology,2018-01-19,OTM,2050.0,20.5,52.5,1609.589722,67


In [3]:
top10 = options_data.groupby('Sym').count().sort_values('OptionType', ascending=False).reset_index()
top10[['Sym','OptionType']].head(10)

,Sym,OptionType
0,AMD,1177
1,MU,729
2,AAPL,700
3,FB,566
4,SNAP,459
5,MSFT,456
6,TWTR,375
7,INTC,375
8,BABA,369
9,JD,324


In [4]:
sym = top10[['Sym']].iloc[:10, 0].tolist()
#filter by Sector
other_symbols = options_data[~options_data['Sym'].isin(sym)].index
options_data.drop(other_symbols, inplace=True)
sym

['AMD', 'MU', 'AAPL', 'FB', 'SNAP', 'MSFT', 'TWTR', 'INTC', 'BABA', 'JD']

In [5]:
def get_stock_data(sym):
  sector_data = pd.DataFrame()
  for i in sym:
    sym_data = pd.read_csv('../Data/Stock/'+ i +'.csv')
    sym_data['Sym'] = i
    sector_data = pd.concat([sector_data, sym_data])
  return sector_data

In [6]:
def get_attention_trend_data(sym):
  attention_data = pd.DataFrame()
  for i in sym:
    sym_data = pd.read_csv('../Data/Attention/'+ i +'_attention.csv')
    sym_data['Sym'] = i
    attention_data = pd.concat([attention_data, sym_data])
  return attention_data

In [7]:
stock_data = get_stock_data(sym)
attention_data =get_attention_trend_data(sym)

In [8]:
attention_data.head()

,date,attention,Sym
0,2016-10-01,44.0,AMD
1,2016-10-02,41.0,AMD
2,2016-10-03,38.0,AMD
3,2016-10-04,41.0,AMD
4,2016-10-05,43.0,AMD


In [9]:
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Sym
0,2016-04-07,2.76,2.83,2.61,2.64,2.64,13479200,AMD
1,2016-04-08,2.70,2.76,2.68,2.74,2.74,8489000,AMD
2,2016-04-11,2.76,2.82,2.74,2.76,2.76,9045100,AMD
3,2016-04-12,2.79,2.85,2.76,2.81,2.81,16131900,AMD
4,2016-04-13,2.78,2.82,2.73,2.80,2.80,8048000,AMD


In [10]:
stock_data['ExpirationDate'] = stock_data['Date'].astype('datetime64[ns]')
attention_data['ExpirationDate'] = attention_data['date'].astype('datetime64[ns]')
stock_data = stock_data.drop('Date', axis=1)
attention_data = attention_data.drop('date', axis=1)
options_data['ExpirationDate'] =options_data['ExpirationDate'].astype('datetime64[ns]')

In [11]:
Tech_data = stock_data.merge(options_data,on=['ExpirationDate', 'Sym'],how='inner')

In [12]:
Tech_data = Tech_data.merge(attention_data,on=['ExpirationDate', 'Sym'],how='inner')

In [13]:
Tech_data

,Open,High,Low,Close,Adj Close,Volume,Sym,ExpirationDate,index,Unnamed: 0,...,Date,Spent,Sector,ChainLocation,Spent/contract,Spent/share,break_even,Contract period in hrs,Contract period in days,attention
0,10.810000,10.970000,10.530000,10.730000,10.730000,42128800,AMD,2017-12-01,42368,42368,...,2017-11-24,22000,Technology,OTM,220.0,2.2,13.7,172.469722,7,48.0
1,10.810000,10.970000,10.530000,10.730000,10.730000,42128800,AMD,2017-12-01,42461,42461,...,2017-11-21,32000,Technology,OTM,320.0,3.2,14.7,247.437778,10,48.0
2,10.810000,10.970000,10.530000,10.730000,10.730000,42128800,AMD,2017-12-01,42511,42511,...,2017-11-21,30000,Technology,OTM,300.0,3.0,14.5,245.110556,10,48.0
3,10.130000,10.320000,10.020000,10.290000,10.290000,45462300,AMD,2017-12-15,40328,40328,...,2018-01-03,34000,Technology,OTM,340.0,3.4,15.4,-448.766111,-19,45.0
4,10.130000,10.320000,10.020000,10.290000,10.290000,45462300,AMD,2017-12-15,40356,40356,...,2018-01-03,119000,Technology,OTM,1190.0,11.9,24.9,-450.706389,-19,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,31.450001,31.690001,30.500000,30.940001,30.940001,13990300,JD,2019-09-20,8951,8951,...,2019-04-17,118000,Technology,OTM,1180.0,11.8,46.8,3749.196389,156,29.0
5525,31.450001,31.690001,30.500000,30.940001,30.940001,13990300,JD,2019-09-20,10024,10024,...,2019-04-04,149000,Technology,OTM,1490.0,14.9,53.9,4061.393056,169,29.0
5526,39.830002,40.384998,39.709999,40.349998,40.349998,9941900,JD,2020-01-17,12719,12719,...,2019-03-01,730000,Technology,ITM,7300.0,73.0,98.0,7735.487500,322,31.0
5527,39.830002,40.384998,39.709999,40.349998,40.349998,9941900,JD,2020-01-17,15633,15633,...,2019-01-24,194000,Technology,OTM,1940.0,19.4,52.4,8597.234722,358,31.0


In [14]:
Tech_data['Profit'] = (Tech_data['break_even'] < Tech_data['Close']) 
S = Tech_data[Tech_data['Profit'] == True]

In [15]:
print("Total number of profitable options: ",S.shape[0])
print("Total number of data entries: ",Tech_data.shape[0])

Total number of profitable options:  104
Total number of data entries:  5529


In [49]:
Tech_data.to_csv('Tech_cleaned.csv')